OBJECTIVE: Create a dataframe of relevant variables using a cohort of patients who would have been eligible for the KEYNOTE-361 and IMvigor130 clinical trials (i.e. patients who received first-line treatment with either atezolizumab, pembrolizumab, or platinum based chemotherapy)

In [252]:
import numpy as np
import pandas as pd

In [253]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

1. Load the full cohort previously defined

In [589]:
cohort = pd.read_csv('full_cohort.csv')

In [590]:
cohort.sample(3)

,PatientID,LineName,StartDate
5419,FAB1E2FFA16A8,chemo,2019-02-28
5725,F5F8FAFD1F1E9,chemo,2014-06-19
5163,FD690D88ADFD9,chemo,2012-02-09


In [591]:
cohort_IDs = cohort['PatientID'].to_numpy()

In [592]:
len(cohort_IDs)

6461

2. Load the demographics file and clean it

In [593]:
demographics = pd.read_csv('../data/Demographics.csv')

In [594]:
demographics.sample(3)

,PatientID,BirthYear,Gender,Race,Ethnicity,State
10414,F800B5E99F845,1959,F,White,Not Hispanic or Latino,MI
349,F66A29469354F,1940,F,Other Race,Not Hispanic or Latino,NaN
9440,F1EA914FCECB7,1970,M,Black or African American,Not Hispanic or Latino,NJ


In [595]:
demographics = demographics[demographics['PatientID'].isin(cohort_IDs)]

In [596]:
row_ID(demographics)

(6461, 6461)

In [597]:
demographics.sample(3)

,PatientID,BirthYear,Gender,Race,Ethnicity,State
5144,F459CDE9E80B8,1936,F,White,Not Hispanic or Latino,FL
5038,FA452B32C1EBE,1950,M,White,Not Hispanic or Latino,CA
6525,FA04ACD3AF21D,1960,M,Other Race,NaN,FL


Race:
The recommendation from Flatiron is to do the following:
This approach specifically addresses the nuance of “Hispanic or Latino” appearing as both a Race and Ethnicity value in Flatiron data, as detailed in the Race and Ethnicity Overview. In order to align with OMB Standards, Flatiron recommends treating “Hispanic or Latino” as an ethnicity, using the following logic:

-Identify patients with a Race value of “Hispanic or Latino”
-For these patients, recode Race to NULL and Ethnicity to “Hispanic or Latino”

The resulting dataset will remove all instances of “Hispanic or Latino” as a Race, leaving “White,” “Black or African American,” “Asian,” “Other Race,” and NULL as possible Race values. 

In [598]:
# If race value is 'Hispanic or Latino', code as unknown, otherwise value unchanged.
demographics['race'] = (
    np.where(demographics['Race'] == 'Hispanic or Latino', 'NULL', demographics['Race'])
)

In [599]:
# Missing race value will be recoded as Unknown
demographics['race'] = demographics['race'].fillna('NULL')

In [600]:
demographics['race'].value_counts().sum()

np.int64(6461)

In [601]:
race_counts = demographics['Race'].value_counts()
print(race_counts)

Race
White                        4561
Other Race                    822
Black or African American     306
Asian                          86
Hispanic or Latino              8
Name: count, dtype: int64


In [602]:
race_counts = demographics['race'].value_counts()
print(race_counts)

race
White                        4561
Other Race                    822
NULL                          686
Black or African American     306
Asian                          86
Name: count, dtype: int64


Ethnicity:

In [603]:
# If race value is equal to 'Hispanic or Latino', code ethnicity as 'Hispanic or Latino', otherwise unchanged. 
demographics['ethnicity'] = (
    np.where(demographics['Race'] == 'Hispanic or Latino', 'hispanic_latino', demographics['Ethnicity'])
)

In [604]:
demographics['ethnicity'] = demographics['ethnicity'].fillna('NULL')

In [605]:
demographics['ethnicity'] = demographics['ethnicity'].replace({'Hispanic or Latino': 'hispanic_latino'})

In [606]:
demographics['ethnicity'] = demographics['ethnicity'].replace({'Not Hispanic or Latino': 'not_hispanic_latino'})

In [607]:
ethnicity_counts = demographics['ethnicity'].value_counts()
print(ethnicity_counts)

ethnicity
not_hispanic_latino    4896
NULL                   1318
hispanic_latino         247
Name: count, dtype: int64


In [608]:
demographics = demographics.drop(columns = ['Race', 'Ethnicity'])

In [609]:
demographics.sample(3)

,PatientID,BirthYear,Gender,State,race,ethnicity
8687,F7A57E62C31F4,1938,M,MI,White,not_hispanic_latino
4052,FC22836CC5D6E,1941,M,AZ,White,not_hispanic_latino
5471,FC31670B9BAF1,1937,F,FL,Other Race,NULL


Per Flatiron, it is recommended that race and ethnicity are combined into a single variable as follows: 
-Hispanic or Latino 
-Not Hispanic or Latino, White 
-Not Hispanic or Latino, Black or African American 
-Not Hispanic or Latino, Asian 
-Not Hispanic or Latino, Other Race 
-Not Hispanic or Latino, Unknown Race 
-Unknown  
But will defer creating this column, given that there is some complexity in deciding how to handle cases where either race or ethnicity is unknown, will defer for now, given that race is unlikely to be central to the question at hand.

Birthyear, convert into Age; use the date of first line start to calculate age

In [610]:
enhanced_adv = pd.read_csv('../data/Enhanced_AdvUrothelial.csv')

In [611]:
demographics = pd.merge(demographics, cohort[['PatientID', 'StartDate']], on = 'PatientID')

In [612]:
demographics.sample(3)

,PatientID,BirthYear,Gender,State,race,ethnicity,StartDate
1892,F74A00D5C4ECE,1936,F,AZ,NULL,NULL,2021-02-26
5421,F637B9562F227,1934,M,PA,White,not_hispanic_latino,2019-08-27
6029,F3ED95E045D22,1944,M,TX,White,not_hispanic_latino,2022-06-15


In [613]:
print(demographics['StartDate'].dtype)

object


In [614]:
print(demographics['StartDate'].isnull().sum())

0


In [615]:
print(demographics['StartDate'].apply(type).unique())

[<class 'str'>]


In [616]:
demographics['StartDate'] = pd.to_datetime(demographics['StartDate'], format="%Y-%m-%d")

In [617]:
print(demographics['StartDate'].dtype)

datetime64[ns]


In [618]:
demographics.loc[:, 'age'] = demographics['StartDate'].dt.year - demographics['BirthYear']

In [619]:
demographics.sample(3)

,PatientID,BirthYear,Gender,State,race,ethnicity,StartDate,age
5214,FA53ADD2356B0,1940,M,OK,White,not_hispanic_latino,2020-10-15,80
118,F1C365C98E575,1941,F,NaN,Other Race,not_hispanic_latino,2020-01-20,79
4260,F06D4B8724238,1952,F,MI,White,hispanic_latino,2020-01-15,68


In [620]:
demographics = demographics.drop(columns = ['BirthYear', 'StartDate'])

In [621]:
demographics.sample(3)

,PatientID,Gender,State,race,ethnicity,age
5388,FB47F7161FDA6,F,PA,White,not_hispanic_latino,54
6118,F708630FFFD6C,M,TX,NULL,NULL,75
2392,F57ECA6012ABE,M,FL,White,not_hispanic_latino,84


Practice type

In [622]:
practice = pd.read_csv('../data/Practice.csv')

In [623]:
practice = practice[practice['PatientID'].isin(cohort_IDs)]

In [624]:
row_ID(practice)

(7036, 6461)

In [625]:
practice_counts = practice['PracticeType'].value_counts()
print(practice_counts)

PracticeType
COMMUNITY    5751
ACADEMIC     1285
Name: count, dtype: int64


In cases where patients have multiple responses for PracticeType, need to address this by labeling as "BOTH"

In [626]:
#First determine how many practice types are present for each patient
practice_unique_count = (
    practice.groupby('PatientID')['PracticeType'].agg('nunique')
    .to_frame()
    .reset_index()
    .rename(columns = {'PracticeType': 'n_type'})
)

In [627]:
practice_n = pd.merge(practice, practice_unique_count, on = 'PatientID')

In [628]:
#Label patients with more than practice type with "BOTH"
practice_n['p_type'] = (
    np.where(practice_n['n_type'] == 1, practice_n['PracticeType'], 'BOTH')
)

In [629]:
practice_n = (
    practice_n.drop_duplicates(subset = ['PatientID'], keep = 'first')
    .filter(items = ['PatientID', 'p_type'])
)

In [630]:
row_ID(practice_n)

(6461, 6461)

In [631]:
practice_n.sample(3)

,PatientID,p_type
139,F44EF2D409AA8,COMMUNITY
1968,FC451996EF9E6,ACADEMIC
4057,F32BDD4E55B6D,BOTH


In [632]:
practice_counts = practice_n['p_type'].value_counts()
print(practice_counts)

p_type
COMMUNITY    5179
ACADEMIC      816
BOTH          466
Name: count, dtype: int64


In [633]:
demographics = pd.merge(demographics, practice_n, on = 'PatientID')

In [634]:
demographics.sample(3)

,PatientID,Gender,State,race,ethnicity,age,p_type
5809,F4A9D954D6409,M,TN,White,not_hispanic_latino,57,COMMUNITY
6443,F9C377B0A7C9A,M,WA,NULL,NULL,58,COMMUNITY
5805,F473BCDD354F8,F,TN,White,not_hispanic_latino,79,COMMUNITY


Gender:

In [635]:
gender_counts = demographics['Gender'].value_counts()
print(gender_counts)

Gender
M    4726
F    1733
Name: count, dtype: int64


Missing values noted, but will avoid imputation for now, with plans to address missingness in ultimate step

In [636]:
#Convert column name to snake
demographics = demographics.rename(columns = {'Gender': 'gender'})

State:

In [637]:
# Group states into Census-Bureau regions  
state_dict = { 
    'ME': 'northeast', 
    'NH': 'northeast',
    'VT': 'northeast', 
    'MA': 'northeast',
    'CT': 'northeast',
    'RI': 'northeast',  
    'NY': 'northeast', 
    'NJ': 'northeast', 
    'PA': 'northeast', 
    'IL': 'midwest', 
    'IN': 'midwest', 
    'MI': 'midwest', 
    'OH': 'midwest', 
    'WI': 'midwest',
    'IA': 'midwest',
    'KS': 'midwest',
    'MN': 'midwest',
    'MO': 'midwest', 
    'NE': 'midwest',
    'ND': 'midwest',
    'SD': 'midwest',
    'DE': 'south',
    'FL': 'south',
    'GA': 'south',
    'MD': 'south',
    'NC': 'south', 
    'SC': 'south',
    'VA': 'south',
    'DC': 'south',
    'WV': 'south',
    'AL': 'south',
    'KY': 'south',
    'MS': 'south',
    'TN': 'south',
    'AR': 'south',
    'LA': 'south',
    'OK': 'south',
    'TX': 'south',
    'AZ': 'west',
    'CO': 'west',
    'ID': 'west',
    'MT': 'west',
    'NV': 'west',
    'NM': 'west',
    'UT': 'west',
    'WY': 'west',
    'AK': 'west',
    'CA': 'west',
    'HI': 'west',
    'OR': 'west',
    'WA': 'west',
    'PR': 'unknown'
}

demographics['region'] = demographics['State'].map(state_dict)

In [638]:
demographics['region'] = demographics['region'].fillna('unknown')

In [639]:
region_counts = demographics['region'].value_counts()
print(region_counts)

region
south        2578
unknown      1523
west          809
northeast     799
midwest       752
Name: count, dtype: int64


In [640]:
demographics = demographics.drop(columns = ['State'])

In [641]:
demographics.sample(3)

,PatientID,gender,race,ethnicity,age,p_type,region
3384,FF368521234C2,M,White,NULL,85,COMMUNITY,south
5077,F610F6D03FD76,F,White,not_hispanic_latino,76,COMMUNITY,midwest
6123,FC90A7BC37991,M,NULL,NULL,61,COMMUNITY,south


In [642]:
%whos DataFrame

Variable                Type         Data/Info
----------------------------------------------
cohort                  DataFrame              PatientID      <...>\n[6461 rows x 3 columns]
demographics            DataFrame              PatientID gende<...>\n[6461 rows x 7 columns]
enhanced_adv            DataFrame               PatientID Diag<...>[13129 rows x 13 columns]
practice                DataFrame               PatientID     <...>\n[7036 rows x 4 columns]
practice_n              DataFrame              PatientID     p<...>\n[6461 rows x 2 columns]
practice_unique_count   DataFrame              PatientID  n_ty<...>\n[6461 rows x 2 columns]


In [643]:
#Keep cohort, demographics, enhanced_adv
del practice
del practice_n
del practice_unique_count

3. Clean enhanced_adv dataset

In [644]:
row_ID(enhanced_adv)

(13129, 13129)

In [645]:
#filter for patients in the cohort
enhanced_adv = enhanced_adv[enhanced_adv['PatientID'].isin(cohort_IDs)]

In [646]:
row_ID(enhanced_adv)

(6461, 6461)

In [647]:
enhanced_adv.sample(3)

,PatientID,DiagnosisDate,AdvancedDiagnosisDate,PrimarySite,DiseaseGrade,GroupStage,TStage,NStage,MStage,SmokingStatus,Surgery,SurgeryDate,SurgeryType
2675,FC5FE9D4A3DF8,2019-05-27,2019-05-27,Renal Pelvis,High grade (G2/G3/G4),Stage IIIA,T3,N2,M0,No history of smoking,True,2019-09-23,Nephroureterectomy
750,FF4A7DFBB013A,2013-09-26,2021-05-05,Bladder,High grade (G2/G3/G4),Unknown/not documented,Tis,N0,M0,History of smoking,True,2014-04-15,Cystoprostatectomy
2424,FDAF4DAA6549F,2012-07-03,2022-01-19,Bladder,High grade (G2/G3/G4),Stage 0a,Ta,N0,M0,History of smoking,True,2020-09-21,Nephroureterectomy


GroupStage

In [648]:
stage_counts = enhanced_adv['GroupStage'].value_counts()
print(stage_counts)

GroupStage
Unknown/not documented    2978
Stage IV                  2055
Stage II                   459
Stage III                  263
Stage IVB                  172
Stage IIIA                 164
Stage IVA                  119
Stage I                    113
Stage IIIB                 104
Stage 0is                   20
Stage 0a                    14
Name: count, dtype: int64


In [649]:
# Dictionary for regrouping stages
stage_dict = { 
    'Stage 0': '0',
    'Stage 0is': '0',
    'Stage 0a': '0',
    'Stage I': 'I',
    'Stage II': 'II',
    'Stage III': 'III',
    'Stage IIIA': 'III',
    'Stage IIIB': 'III',
    'Stage IV': 'IV',
    'Stage IVA': 'IV',
    'Stage IVB': 'IV',
    'Unknown/not documented': 'unknown'
}

In [650]:
enhanced_adv['stage'] = enhanced_adv['GroupStage'].map(stage_dict)

In [651]:
stage_counts = enhanced_adv['stage'].value_counts()
print(stage_counts)

stage
unknown    2978
IV         2346
III         531
II          459
I           113
0            34
Name: count, dtype: int64


In [652]:
enhanced_adv = enhanced_adv.drop(columns = ['GroupStage'])

AdvancedDiagnosisDate

In [653]:
enhanced_adv = enhanced_adv.rename(columns = {'AdvancedDiagnosisDate': 'adv_diagnosis_date'})

In [654]:
enhanced_adv['adv_diagnosis_date'] = pd.to_datetime(enhanced_adv['adv_diagnosis_date'], format="%Y-%m-%d")

In [655]:
#confirm datetime conversion successful
print(enhanced_adv['adv_diagnosis_date'].dtype)

datetime64[ns]


In [656]:
enhanced_adv.loc[:, 'adv_diagnosis_year'] = enhanced_adv['adv_diagnosis_date'].dt.year

In [657]:
enhanced_adv.sample(3)

,PatientID,DiagnosisDate,adv_diagnosis_date,PrimarySite,DiseaseGrade,TStage,NStage,MStage,SmokingStatus,Surgery,SurgeryDate,SurgeryType,stage,adv_diagnosis_year
9036,FCD58A84D0899,2018-09-06,2018-09-06,Renal Pelvis,High grade (G2/G3/G4),Unknown/not documented,Unknown/not documented,M1,History of smoking,True,2019-03-28,Nephroureterectomy,IV,2018
7776,F7A207FC3A403,2004-10-01,2012-02-24,Bladder,High grade (G2/G3/G4),T1,Unknown/not documented,Unknown/not documented,History of smoking,False,NaN,NaN,unknown,2012
5680,FE97CAE1D4D48,2018-02-21,2020-06-17,Bladder,High grade (G2/G3/G4),T3b,N0,M0,History of smoking,True,2018-04-10,Other,unknown,2020


DiagnosisDate

In [658]:
enhanced_adv = enhanced_adv.rename(columns = {'DiagnosisDate': 'diagnosis_date'})

In [659]:
enhanced_adv['diagnosis_date'] = pd.to_datetime(enhanced_adv['diagnosis_date'], format="%Y-%m-%d")

In [660]:
# Missing diagnosis_date will be replaced with adv_date; other dates will be left untouched. 
enhanced_adv['diagnosis_date'] = (
    np.where(enhanced_adv['diagnosis_date'].isna(), enhanced_adv['adv_diagnosis_date'], enhanced_adv['diagnosis_date'])
)

In [661]:
#confirm datetime conversion successful
print(enhanced_adv['adv_diagnosis_date'].dtype)

datetime64[ns]


Time from diagnosis date to metastatic date

In [662]:
enhanced_adv.loc[:, 'delta_adv_diagnosis'] = (enhanced_adv['adv_diagnosis_date'] - enhanced_adv['diagnosis_date']).dt.days

In [663]:
enhanced_adv.sample(3)

,PatientID,diagnosis_date,adv_diagnosis_date,PrimarySite,DiseaseGrade,TStage,NStage,MStage,SmokingStatus,Surgery,SurgeryDate,SurgeryType,stage,adv_diagnosis_year,delta_adv_diagnosis
7154,F2FB8F774FC1D,2013-10-02,2015-02-02,Ureter,High grade (G2/G3/G4),T3,N0,M0,No history of smoking,True,2013-10-25,Nephroureterectomy,unknown,2015,488
8566,F87589A155D0C,2022-05-09,2022-05-09,Bladder,High grade (G2/G3/G4),T2,N0,M1b,History of smoking,False,NaN,NaN,IV,2022,0
6182,F233DF7C8E89C,2022-10-14,2022-10-14,Bladder,Unknown/not documented,Unknown/not documented,Unknown/not documented,M1,History of smoking,False,NaN,NaN,IV,2022,0


PrimarySite

In [664]:
site_counts = enhanced_adv['PrimarySite'].value_counts()
print(site_counts)

PrimarySite
Bladder         4919
Renal Pelvis     897
Ureter           599
Urethra           46
Name: count, dtype: int64


In [665]:
enhanced_adv = enhanced_adv.rename(columns = {'PrimarySite': 'primary_site'})

In [666]:
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Bladder': 'lower_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Urethra': 'lower_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Renal Pelvis': 'upper_tract'})
enhanced_adv['primary_site'] = enhanced_adv['primary_site'].replace({'Ureter': 'upper_tract'})

In [667]:
site_counts = enhanced_adv['primary_site'].value_counts()
print(site_counts)

primary_site
lower_tract    4965
upper_tract    1496
Name: count, dtype: int64


DiseaseGrade

In [668]:
enhanced_adv['DiseaseGrade'].value_counts()

DiseaseGrade
High grade (G2/G3/G4)     5441
Unknown/not documented     711
Low grade (G1)             309
Name: count, dtype: int64

In [669]:
enhanced_adv = enhanced_adv.rename(columns = {'DiseaseGrade': 'disease_grade'})

In [670]:
enhanced_adv['disease_grade'] = enhanced_adv['disease_grade'].replace({'High grade (G2/G3/G4)': 'high_grade'})
enhanced_adv['disease_grade'] = enhanced_adv['disease_grade'].replace({'Unknown/not documented': 'unknown'})
enhanced_adv['disease_grade'] = enhanced_adv['disease_grade'].replace({'Low grade (G1)': 'low_grade'})

In [671]:
enhanced_adv['disease_grade'].value_counts()

disease_grade
high_grade    5441
unknown        711
low_grade      309
Name: count, dtype: int64

TStage

In [672]:
enhanced_adv['TStage'].value_counts()

TStage
Unknown/not documented    2226
T2                        1140
T3                         738
T1                         720
T4                         295
T4a                        232
T3a                        200
T3b                        196
T2b                        192
T2a                        158
TX                         136
Ta                         126
T4b                         54
Tis                         42
T0                           6
Name: count, dtype: int64

In [673]:
enhanced_adv = enhanced_adv.rename(columns = {'TStage': 't_stage'})

In [674]:
# Dictionary for regrouping t stages
t_stage_dict = { 
    'Unknown/not documented': 'unknown',
    'T2': 'T2',
    'T3': 'T3',
    'T1': 'T1',
    'T4': 'T4',
    'T4a': 'T4',
    'T3a': 'T3',
    'T3b': 'T3',
    'T2b': 'T2',
    'T2a': 'T2',
    'TX': 'unknown',
    'Ta': 'Ta',
    'T4b': 'T4',
    'Tis': 'Tis',
    'T0': 'T0'
    
}

enhanced_adv['t_stage'] = enhanced_adv['t_stage'].map(t_stage_dict)

In [675]:
enhanced_adv['t_stage'].value_counts()

t_stage
unknown    2362
T2         1490
T3         1134
T1          720
T4          581
Ta          126
Tis          42
T0            6
Name: count, dtype: int64

NStage

In [676]:
enhanced_adv['NStage'].value_counts()

NStage
Unknown/not documented    3107
N0                        1315
NX                         743
N2                         612
N1                         497
N3                         187
Name: count, dtype: int64

In [677]:
enhanced_adv = enhanced_adv.rename(columns = {'NStage': 'n_stage'})

In [678]:
# Dictionary for regrouping n stages
n_stage_dict = { 
    'Unknown/not documented': 'unknown',
    'N0': 'N0',
    'NX': 'unknown',
    'N2': 'N2',
    'N1': 'N1',
    'N3': 'N3'
}

enhanced_adv['n_stage'] = enhanced_adv['n_stage'].map(n_stage_dict)

In [679]:
enhanced_adv['n_stage'].value_counts()

n_stage
unknown    3850
N0         1315
N2          612
N1          497
N3          187
Name: count, dtype: int64

MStage

In [680]:
enhanced_adv['MStage'].value_counts()

MStage
M0                        2630
Unknown/not documented    1878
M1                        1330
MX                         393
M1b                        139
M1a                         91
Name: count, dtype: int64

In [681]:
enhanced_adv = enhanced_adv.rename(columns = {'MStage': 'm_stage'})

In [682]:
# Dictionary for regrouping m stages
m_stage_dict = { 
    'M0': 'M0',
    'Unknown/not documented': 'unknown',
    'M1': 'M1',
    'MX': 'unknown',
    'M1b': 'M1',
    'M1a': 'M1'
}

enhanced_adv['m_stage'] = enhanced_adv['m_stage'].map(m_stage_dict)

In [683]:
enhanced_adv['m_stage'].value_counts()

m_stage
M0         2630
unknown    2271
M1         1560
Name: count, dtype: int64

SmokingStatus

In [684]:
enhanced_adv['SmokingStatus'].value_counts()

SmokingStatus
History of smoking        4742
No history of smoking     1677
Unknown/not documented      42
Name: count, dtype: int64

In [685]:
enhanced_adv = enhanced_adv.rename(columns = {'SmokingStatus': 'smoking_status'})

In [686]:
enhanced_adv['smoking_status'] = enhanced_adv['smoking_status'].replace({'History of smoking': 'smoker'})
enhanced_adv['smoking_status'] = enhanced_adv['smoking_status'].replace({'No history of smoking': 'never_smoker'})
enhanced_adv['smoking_status'] = enhanced_adv['smoking_status'].replace({'Unknown/not documented': 'unknown'})

In [687]:
enhanced_adv['smoking_status'].value_counts()

smoking_status
smoker          4742
never_smoker    1677
unknown           42
Name: count, dtype: int64

Surgery

In [688]:
enhanced_adv['Surgery'].value_counts()

Surgery
False    3345
True     3116
Name: count, dtype: int64

In [689]:
enhanced_adv = enhanced_adv.rename(columns = {'Surgery': 'surgery_status'})

In [690]:
enhanced_adv['surgery_status'].value_counts()

surgery_status
False    3345
True     3116
Name: count, dtype: int64

In [691]:
print(enhanced_adv['surgery_status'].dtype)

bool


SurgeryDate

In [692]:
print(enhanced_adv['SurgeryDate'].dtype)

object


In [693]:
enhanced_adv = enhanced_adv.rename(columns = {'SurgeryDate': 'surgery_date'})

In [694]:
enhanced_adv['surgery_date'] = pd.to_datetime(enhanced_adv['surgery_date'], format="%Y-%m-%d")

In [695]:
print(enhanced_adv['surgery_date'].dtype)

datetime64[ns]


Leaving cases where there is no surgery date, empty for now, preserving date formatting

SurgeryType

In [696]:
enhanced_adv['SurgeryType'].value_counts()

SurgeryType
Cystoprostatectomy               1210
Nephroureterectomy                882
Complete (radical) cystectomy     539
Partial cystectomy                126
Nephrectomy                       124
Ureterectomy                       98
Cystectomy, NOS                    73
Other                              58
Unknown/not documented              4
Urethrectomy                        2
Name: count, dtype: int64

In [697]:
enhanced_adv = enhanced_adv.rename(columns = {'SurgeryType': 'surgery_type'})

In [698]:
# Dictionary for regrouping surgery type
surgery_type_dict = { 
    'Cystoprostatectomy': 'cystoprostatectomy',
    'Nephroureterectomy': 'nephroureterectomy',
    'Complete (radical) cystectomy': 'radical_cystectomy',
    'Partial cystectomy': 'partial_cystectomy',
    'Nephrectomy': 'nephrectomy',
    'Ureterectomy': 'ureterectomy',
    'Cystectomy, NOS': 'cystectomy_nos',
    'Other': 'other_surgery',
    'Unknown/not documented': 'unknown_surgery',
    'Urethrectomy': 'urethrectomy'
}

enhanced_adv['surgery_type'] = enhanced_adv['surgery_type'].map(surgery_type_dict)

In [699]:
enhanced_adv['surgery_type'] = enhanced_adv['surgery_type'].fillna('no_surgery')

In [700]:
enhanced_adv['surgery_type'].value_counts()

surgery_type
no_surgery            3345
cystoprostatectomy    1210
nephroureterectomy     882
radical_cystectomy     539
partial_cystectomy     126
nephrectomy            124
ureterectomy            98
cystectomy_nos          73
other_surgery           58
unknown_surgery          4
urethrectomy             2
Name: count, dtype: int64

In [701]:
#Final enhanced_adv dataframe
enhanced_adv.sample(3)

,PatientID,diagnosis_date,adv_diagnosis_date,primary_site,disease_grade,t_stage,n_stage,m_stage,smoking_status,surgery_status,surgery_date,surgery_type,stage,adv_diagnosis_year,delta_adv_diagnosis
6116,FB82ACEC607C6,2022-02-09,2022-02-09,lower_tract,high_grade,unknown,unknown,M1,never_smoker,False,NaT,no_surgery,IV,2022,0
10221,FB9A9E2A69639,2012-12-12,2012-12-12,lower_tract,high_grade,T4,unknown,unknown,smoker,False,NaT,no_surgery,IV,2012,0
11915,F4A9D954D6409,2020-10-12,2020-10-12,upper_tract,high_grade,T4,N1,M0,smoker,True,2020-11-20,nephroureterectomy,IV,2020,0
